# Inspect XRFI Files

In [1]:
import numpy as np
import os as os
import h5py

from ipywidgets import interact

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# configure matplotlib
mpl.style.use("classic")

In [5]:
data_path = "/lustre/aoc/projects/hera/pkeller/data/H1C_IDR3.2/sample/EQ28_FA_B2.h5"

with h5py.File(data_path, "r") as f:
    jdlist = f["JD"][()]
    lst = f["LST"][()]
    frq = f["FRQ"][()]

In [6]:
rdir = "/lustre/aoc/projects/hera/H1C_IDR3/IDR3_2/"

flg_list = []
jd = []

for j in jdlist:
    fname = f"zen.{j}.total_threshold_and_a_priori_flags.h5"
    rfi_path = os.path.join(rdir, str(j), fname)
    
    if os.path.exists(rfi_path):
        with h5py.File(rfi_path, "r") as f:
            flags = f["Data/flag_array"][:, :, 0]
            lstd = f["Header/lst_array"][()] * 12 / np.pi
            flg_list.append(flags[np.where((lstd > min(lst)) & (lstd < max(lst)))])
        jd.append(j)

flags = np.asarray(flg_list)
print(flags.shape)

(34, 834, 1024)


In [7]:
def plot(j):
    fig, ax = plt.subplots(1, 1, figsize=(6.5, 5))
    ax.imshow(flags[j][:, 535:696], cmap=cm.bone, aspect="auto", interpolation="None", extent=(np.amin(frq), np.amax(frq), np.max(lst), np.min(lst)))
    plt.setp(ax, xlabel="Frequency (MHz)", ylabel="LST (h)", ylim=[max(lst), min(lst)])
    plt.title(f"JD={jd[j]}")
    plt.show()

interact(plot, j=(0, flags.shape[0]-1, 1))

interactive(children=(IntSlider(value=16, description='j', max=33), Output()), _dom_classes=('widget-interact'…

<function __main__.plot(j)>